In [1]:
import pandas as pd
import os
import sqlalchemy
import pyodbc
from datetime import datetime, timedelta
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn import tree
import sklearn
from imblearn.over_sampling import SMOTE, SMOTENC
from forex_python.converter import CurrencyRates
from time import sleep
import matplotlib.pyplot as plt
from sklearn.tree import export_graphviz
import time

In [2]:
# libraries = ['pd', 'sqlalchemy', 'pyodbc', 'np', 'sklearn', 'imblearn']
# def get_version(lib):
#     lib = eval(lib)
#     try:
#         return lib.__version__
#     except AttributeError:
#         return "not found"
    
# for lib in libraries:
#     version = get_version(lib)
#     print(f"{lib}=={version}")

# Объединяем все Excel файлы в 1 датасет

In [3]:
# Имена папок
folders = ["2020.12","2021.01","2021.02","2021.03","2021.04","2021.05",
           "2021.06","2021.07","2021.08","2021.09","2021.10","2021.11","2021.12","2022.01"]

In [9]:
# Пустой список путей + даты
file_paths = []
# Для каждой папки из списка папок
for folder in folders:
    # Лепим путь к файлу
    folder_path = os.path.join(r"C:\Users\MikheevMN\Эшелоны", folder)
    # folder_path = os.path.join(r"C:\Users\mikhe\код эшелоны\echelons\Эшелоны", folder)
    # Достаем какие файлы в папке
    files = os.listdir(folder_path)
    # Для файла из папки
    for file in files:
        # Если файл не временный то добавляем путь в список путей с датой в конце
        if not file.startswith('~$'):
            file_paths.append((os.path.join(folder_path, file), folder))
            
for file in file_paths:
    print(file)

('C:\\Users\\mikhe\\код эшелоны\\echelons\\Эшелоны\\2020.12\\Портфель цб 12 2020 + об_эшелон.xlsx', '2020.12')
('C:\\Users\\mikhe\\код эшелоны\\echelons\\Эшелоны\\2021.01\\Портфель цб 01 21.xlsm', '2021.01')
('C:\\Users\\mikhe\\код эшелоны\\echelons\\Эшелоны\\2021.02\\Портфель цб 02 21 копия.xlsm', '2021.02')
('C:\\Users\\mikhe\\код эшелоны\\echelons\\Эшелоны\\2021.03\\Портфель цб 03 21 к.xlsm', '2021.03')
('C:\\Users\\mikhe\\код эшелоны\\echelons\\Эшелоны\\2021.04\\Копия Портфель цб 04 21-2.xlsm', '2021.04')
('C:\\Users\\mikhe\\код эшелоны\\echelons\\Эшелоны\\2021.05\\Портфель цб 05 21.xlsm', '2021.05')
('C:\\Users\\mikhe\\код эшелоны\\echelons\\Эшелоны\\2021.06\\Портфель цб 06 21.xlsm', '2021.06')
('C:\\Users\\mikhe\\код эшелоны\\echelons\\Эшелоны\\2021.07\\Портфель цб 07 21.xlsm', '2021.07')
('C:\\Users\\mikhe\\код эшелоны\\echelons\\Эшелоны\\2021.08\\Портфель цб 08 21.xlsm', '2021.08')
('C:\\Users\\mikhe\\код эшелоны\\echelons\\Эшелоны\\2021.09\\Портфель цб 09 21.xlsm', '2021.09')


In [10]:
# Пустой датасет для хранения
combined_data = pd.DataFrame()

In [12]:
column_names = ["Index", "Description", "Duration", "Amount", "Currency", "TRPS", "Rating", "Echelon"]
# Для пути и даты в списке
for file_path, folder in file_paths:
    excel_data = pd.read_excel(file_path, sheet_name="Эшелоны результат", usecols="A:H", header=None)
    excel_data.columns = column_names
    excel_data["Date"] = folder
    combined_data = combined_data.append(excel_data, ignore_index=True)
    # combined_data = pd.concat([combined_data, excel_data], ignore_index=True) 
combined_data

,Index,Description,Duration,Amount,Currency,TRPS,Rating,Echelon,Date
0,ISIN,Description,Дюрация,Amount Outstanding,Currency,TRPS Score,Рейтинг Буква,Eshelon,2020.12
1,XS2134628069,MMCFN 2.550 09/11/25 '25,4.223,500000000,USD,10,BBB-,1,2020.12
2,XS1449458915,GOTRL 5.950 07/19/21,0.558,350000000,USD,10,BB,2,2020.12
3,XS1319822752,EVRE 8.250 01/28/21,0.097,750000000,USD,10,BB+,1,2020.12
4,XS1433454243,FASPM 5.375 06/16/23,2.358,900000000,USD,10,BB,2,2020.12
...,...,...,...,...,...,...,...,...,...
2089,0,GRUPP 06/27/21,Internal error,0,RUB,NaN,NaN,"Ошибка, не численные значения",2022.01
2090,0,GRUPP 06/27/21,Internal error,0,RUB,NaN,NaN,"Ошибка, не численные значения",2022.01
2091,0,GRUPP 06/27/21,Internal error,0,RUB,NaN,NaN,"Ошибка, не численные значения",2022.01
2092,0,GRUPP 06/27/21,Internal error,0,RUB,NaN,NaN,"Ошибка, не численные значения",2022.01


In [13]:
# Выбираем только те строки, где столбец Эшелон != Ошибка, не численные значения
combined_data = combined_data[~combined_data["Echelon"].astype(str).str.contains("Ошибка, не численные значения")]

In [14]:
# Убираем дубликаты строк(при отборе исключаем Date)
combined_data.drop_duplicates(subset=combined_data.columns.difference(["Date"]), inplace=True)

C:\Users\mikhe\AppData\Local\Temp\ipykernel_1672\1773333570.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data.drop_duplicates(subset=combined_data.columns.difference(["Date"]), inplace=True)


In [15]:
# Убираем строки с Null
combined_data.dropna(inplace=True)

C:\Users\mikhe\AppData\Local\Temp\ipykernel_1672\427361800.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data.dropna(inplace=True)


In [16]:
# Преобразуем Date в datetime
combined_data["Date"] = pd.to_datetime(combined_data["Date"], format='%Y.%m')

C:\Users\mikhe\AppData\Local\Temp\ipykernel_1672\1602588503.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data["Date"] = pd.to_datetime(combined_data["Date"], format='%Y.%m')


In [17]:
# Удаляем текстовые TRPS данные
combined_data['TRPS'] = pd.to_numeric(combined_data['TRPS'], errors='coerce')
combined_data.dropna(subset=['TRPS'], inplace=True)

C:\Users\mikhe\AppData\Local\Temp\ipykernel_1672\3276972932.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data['TRPS'] = pd.to_numeric(combined_data['TRPS'], errors='coerce')
C:\Users\mikhe\AppData\Local\Temp\ipykernel_1672\3276972932.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data.dropna(subset=['TRPS'], inplace=True)


In [18]:
# Меняем тип данных и обновляем индексы
combined_data['Duration'] = combined_data['Duration'].astype(float)
combined_data['Amount'] = combined_data['Amount'].astype(float)
combined_data['Echelon'] = combined_data['Echelon'].astype(int)
combined_data.reset_index(drop=True, inplace=True)

C:\Users\mikhe\AppData\Local\Temp\ipykernel_1672\1132213808.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data['Duration'] = combined_data['Duration'].astype(float)
C:\Users\mikhe\AppData\Local\Temp\ipykernel_1672\1132213808.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data['Amount'] = combined_data['Amount'].astype(float)
C:\Users\mikhe\AppData\Local\Temp\ipykernel_1672\1132213808.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [19]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988 entries, 0 to 987
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Index        988 non-null    object        
 1   Description  988 non-null    object        
 2   Duration     988 non-null    float64       
 3   Amount       988 non-null    float64       
 4   Currency     988 non-null    object        
 5   TRPS         988 non-null    float64       
 6   Rating       988 non-null    object        
 7   Echelon      988 non-null    int32         
 8   Date         988 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int32(1), object(4)
memory usage: 65.7+ KB


In [20]:
# Букавы рейтинг меняем на машина понимать язык
replacement_dict = {
    'B-': 1,
    'B': 2,
    'B+': 3,
    'BB-': 4,
    'BB': 5,
    'BB+': 6,
    'BBB-': 7,
    'BBB': 8,
    'BBB+': 9,
    'A-': 10
}
combined_data['Rating'] = combined_data['Rating'].replace(replacement_dict)

C:\Users\mikhe\AppData\Local\Temp\ipykernel_1672\2689714923.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_data['Rating'] = combined_data['Rating'].replace(replacement_dict)
C:\Users\mikhe\AppData\Local\Temp\ipykernel_1672\2689714923.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data['Rating'] = combined_data['Rating'].replace(replacement_dict)


In [21]:
set(combined_data['Rating'])

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

In [22]:
# combined_data.to_csv('echelons.csv', index=False)

In [23]:
# combined_data = pd.read_csv('echelons.csv')
# combined_data["Date"] = pd.to_datetime(combined_data["Date"])
# combined_data.head()

In [24]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988 entries, 0 to 987
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Index        988 non-null    object        
 1   Description  988 non-null    object        
 2   Duration     988 non-null    float64       
 3   Amount       988 non-null    float64       
 4   Currency     988 non-null    object        
 5   TRPS         988 non-null    float64       
 6   Rating       988 non-null    int64         
 7   Echelon      988 non-null    int32         
 8   Date         988 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int32(1), int64(1), object(3)
memory usage: 65.7+ KB


# Достаем из БД уровень листинга бумаги

In [135]:
def query_db(securityid, date):
    arr = 0
    connection_string = r'Driver={SQL Server};Server=DEVELOPERS-SQL;Database=InnerData;Trusted_Connection=yes'
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()
    
    interval_start = date - timedelta(days=100)
                                      
    # тут важно!!! USA 🦅 формат год/день/месяц
    date = date.strftime("%Y-%d-%m")
    interval_start = interval_start.strftime("%Y-%d-%m")
    
    query = f"""
    SELECT TOP 1 ListName
    FROM MM00001_SEM21
    WHERE SecurityId = CONVERT(VARCHAR, ?) AND TradeDate >= ? AND TradeDate < ?
    ORDER BY TradeDate DESC
    """
    
    cursor.execute(query, (securityid, interval_start, date))
    
    arr = cursor.fetchall()
    
    cursor.close()
    connection.close()
    return arr

In [142]:
# Достаем уровень листинга из бд и создаем список
new_column = []
for index, row in combined_data.iterrows():
    list_name = query_db(row['Index'], row['Date'])
    new_column.append(list_name)

In [182]:
new_column[:5]

[[(3, )], [(3, )], [(-1, )], [(3, )], []]

In [183]:
# Делаем список одномерным
new_column_flat = []
for sublist in new_column:
    if sublist == []:
        new_column_flat.append(None)
    else:
        for item in sublist:
            new_column_flat.append(item[0])           
new_column_flat[:5]

[3, 3, -1, 3, None]

In [181]:
len(new_column_flat)

988

In [184]:
# Создаем новый столбец
combined_data['Listing'] = new_column_flat

In [192]:
# Удаляем NaN значения, обнуляем индекс
combined_data = combined_data.dropna()
combined_data = combined_data.reset_index(drop=True)

# Ломбардный список

In [3]:
import xml.etree.ElementTree as ET

In [51]:
folder = ["2020-12-01","2021-01-01","2021-02-01","2021-03-01","2021-04-01","2021-05-01",
           "2021-06-01","2021-07-01","2021-08-01","2021-09-01","2021-10-01","2021-11-01","2021-12-01","2022-01-01"]

In [91]:
# XML
combined_data["Lombard"] = False
# Для каждого файла из папки
for file in folder:
    # Лепим путь к файлу
    file_path = os.path.join(r"C:\Users\MikheevMN\lombard_xml", file)
    file_path = file_path + ".xml"
    tree = ET.parse(file_path)
    tree = ET.parse(xml_path)
    root = tree.getroot()
    # Список исинов
    isin_list = []
    for emitent in root.findall("./Papers/Emitent"):
        for paper in emitent.findall("Paper"):
            isin = paper.get("ISIN")
            mask = (combined_data['Date'] == file) & (combined_data['Index'] == isin)
            if mask.any():
                combined_data.loc[mask, 'Lombard'] = True

In [89]:
# OFZ
ofz_xlsx = r"C:\Users\MikheevMN\bondsearch_26_04_2024.xlsx"
xlsx_data = pd.read_excel(ofz_xlsx, usecols=[18])
for index, row in xlsx_data.iterrows():
    mask = combined_data['Index'] == row['ISIN']
    if mask.any():
        combined_data.loc[mask, 'Lombard'] = True

In [92]:
combined_data['Lombard'] = combined_data['Lombard'].replace({True: 1, False: 0})

In [96]:
# combined_data.to_csv('echelons.csv', index=False)

# Ликвидность

## Создал таблицу в бд с курсами валют для разных дат

In [19]:
# df_ex_rate = pd.read_excel(r'C:\Users\MikheevMN\indexes_archive_20240427.xlsx')
# df_ex_rate["Дата"] = pd.to_datetime(df_ex_rate["Дата"], format="%d.%m.%Y")

In [36]:
# df_ex_rate['RUR'] = 1
# df_ex_rate_melted = pd.melt(df_ex_rate, id_vars=['Дата'], var_name='Валюта', value_name='Курс')
# df_ex_rate_melted['Валюта'] = df_ex_rate_melted['Валюта'].apply(lambda x: 'USD' if x == 'USD/RUB (FX)' else 'EUR' if x == 'EUR/RUB (FX)' else 'RUR')
# df_ex_rate_melted.at[966, 'Курс'] = 87.77
# df_ex_rate_melted[df_ex_rate_melted['Дата']=='2021-07-21']

,Дата,Валюта,Курс
385,2021-07-21,USD,73.87
966,2021-07-21,EUR,87.77
1547,2021-07-21,RUR,1.00


## Vi сделано

In [50]:
# В одну дату может быть 2 записи: в валюте и в рублях

In [10]:
def query_db_vi(securityid, date):
    arr = 0
    connection_string = r'Driver={SQL Server};Server=DEVELOPERS-SQL;Database=InnerData;Trusted_Connection=yes'
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()
    
    interval_start = date - timedelta(days=300)
                                      
    # тут важно!!! USA 🦅 формат год/день/месяц
    date = date.strftime("%Y-%d-%m")
    interval_start = interval_start.strftime("%Y-%d-%m")
    
    query = f"""
    SELECT ROUND(AVG(AmountInDay), 1)
    FROM(
        SELECT TOP 30 TradeDate, SUM(Amount) AS AmountInDay
        FROM(
            SELECT DISTINCT TradeDate, CurrencyId, Value * Курс AS Amount
            FROM MM00001_SEM21 sem LEFT JOIN Exchange_rate ex
                ON ex.Валюта = sem.CurrencyId AND ex.Дата = sem.TradeDate
            WHERE SecurityId = CONVERT(VARCHAR, ?) AND TradeDate >= ? AND TradeDate < ? AND (BoardId = 'TQCB' OR BoardId = 'TQOD' OR BoardId = 'TQOE' OR BoardId = 'TQOY')
            ) AS subquery
        GROUP BY TradeDate
        ORDER BY TradeDate DESC) AS subquery2
    """
    
    cursor.execute(query, (securityid, interval_start, date))
    
    arr = cursor.fetchall()
    
    cursor.close()
    connection.close()
    return arr

In [11]:
query_db_vi('XS2134628069', combined_data['Date'][1])

[(105811.5, )]

## Рассчет V

In [12]:
# пока что удалил дату начала, просто беру топ 30
def query_db_v(date):
    arr = 0
    connection_string = r'Driver={SQL Server};Server=DEVELOPERS-SQL;Database=InnerData;Trusted_Connection=yes'
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()
    
    interval_start = date - timedelta(days=150)
                                      
    # тут важно!!! ПОЧЕМУ ТО ТЕПЕРЬ РАБОТАЕТ гггг.мм.дд UPD опять ддмм
    date = date.strftime("%Y-%d-%m")
    interval_start = interval_start.strftime("%Y-%d-%m")
    
    query = f"""
    SELECT ROUND(AVG(AmountDay), 0)
    FROM(
        SELECT TOP 30 TradeDate, SUM(Amount) AS AmountDay
        FROM(
            SELECT DISTINCT TradeDate, CurrencyId, SecurityId, Value * Курс AS Amount
            FROM MM00001_SEM21 sem LEFT JOIN Exchange_rate ex
                 ON ex.Валюта = sem.CurrencyId AND ex.Дата = sem.TradeDate
            WHERE TradeDate >= ? AND TradeDate < ? AND EngType = 'bn' AND (BoardId = 'TQCB' OR BoardId = 'TQOD' OR BoardId = 'TQOE' OR BoardId = 'TQOY')
            ) AS subquery
        GROUP BY TradeDate
        ORDER BY TradeDate DESC) AS subquery2
    """
    
    cursor.execute(query, (interval_start, date))
    
    arr = cursor.fetchall()
    
    cursor.close()
    connection.close()
    return arr

In [13]:
query_db_v(combined_data['Date'][1])

[(4770814893.0, )]

## Рассчет NTi

In [14]:
def query_db_nti(securityid, date):
    arr = 0
    connection_string = r'Driver={SQL Server};Server=DEVELOPERS-SQL;Database=InnerData;Trusted_Connection=yes'
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()
    
    interval_start = date - timedelta(days=300)
                                      
    # тут важно!!! американский формат год/день/месяц
    date = date.strftime("%Y-%d-%m")
    interval_start = interval_start.strftime("%Y-%d-%m")
    
    query = f"""
    SELECT ROUND(AVG(convert(float, TradesInDay)), 3)
    FROM(
        SELECT TOP 30 TradeDate, SUM(NumTrades) AS TradesInDay
        FROM(
            SELECT DISTINCT TradeDate, Value, NumTrades
            FROM MM00001_SEM21
            WHERE SecurityId = CONVERT(VARCHAR, ?) AND TradeDate >= ? AND TradeDate < ? AND (BoardId = 'TQCB' OR BoardId = 'TQOD' OR BoardId = 'TQOE' OR BoardId = 'TQOY')
            ) AS subquery
        GROUP BY TradeDate
        ORDER BY TradeDate DESC) AS subquery2
    """
    
    cursor.execute(query, (securityid, interval_start, date))
    
    arr = cursor.fetchall()
    
    cursor.close()
    connection.close()
    return arr

In [15]:
query_db_nti('XS1449458915', combined_data['Date'][1])

[(0.567, )]

## Рассчет NT

In [16]:
# пока что удалил дату начала, просто беру топ 30
def query_db_nt(date):
    arr = 0
    connection_string = r'Driver={SQL Server};Server=DEVELOPERS-SQL;Database=InnerData;Trusted_Connection=yes'
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()
    
    interval_start = date - timedelta(days=150)
                                      
    # тут важно!!! ПОЧЕМУ ТО ТЕПЕРЬ РАБОТАЕТ гггг.мм.дд UPD опять ддмм
    date = date.strftime("%Y-%d-%m")
    interval_start = interval_start.strftime("%Y-%d-%m")
    
    query = f"""
    SELECT AVG(TradesInDay)
    FROM(
        SELECT TOP 30 TradeDate, SUM(NumTrades) AS TradesInDay
        FROM(
            SELECT DISTINCT TradeDate, CurrencyId, SecurityId, Value, NumTrades
            FROM MM00001_SEM21
            WHERE TradeDate >= ? AND TradeDate < ? AND EngType = 'bn' AND (BoardId = 'TQCB' OR BoardId = 'TQOD' OR BoardId = 'TQOE' OR BoardId = 'TQOY')
            ) AS subquery
        GROUP BY TradeDate
        ORDER BY TradeDate DESC) AS suquery2
    """
    
    cursor.execute(query, (interval_start, date))
    
    arr = cursor.fetchall()
    
    cursor.close()
    connection.close()
    return arr

In [17]:
query_db_nt(combined_data['Date'][1])

[(32507, )]

## Соединяем все

In [38]:
# df = pd.DataFrame(li_base_list_nach)
# df.to_csv('li_base.csv')

In [4]:
li_base = pd.read_csv('li_base.csv')
li_base_list_nach = li_base['0'].tolist()

FileNotFoundError: [Errno 2] No such file or directory: 'li_base.csv'

In [18]:
li_base_column = []
for index, row in combined_data[715:].iterrows():
    # константы
    a = 0.03
    b = 0.07
    # все переменные, идут запросы к бд
    v_i = query_db_vi(row['Index'], row['Date'])[0][0]
    v = query_db_v(row['Date'])[0][0]
    nt_i = query_db_vi(row['Index'], row['Date'])[0][0]
    nt = query_db_v(row['Date'])[0][0]
    
    # формула
    if v_i == None or v == None or nt_i == None or nt == None:
        li_base = None
    else: 
        li_base = pow(v_i/v, a) * pow(nt_i/nt, b)
    li_base_column.append(li_base)

In [39]:
len(li_base_list_nach)

880

In [40]:
combined_data['Liquidity'] = li_base_list_nach
combined_data = combined_data.fillna(0)
combined_data.head()

,Index,Description,Duration,Amount,Currency,TRPS,Rating,Echelon,Date,Listing,Lombard,Liquidity
0,XS2134628069,MMCFN 2.550 09/11/25 '25,4.223,500000000.0,USD,10.0,7,1,2020-12-01,3.0,0,0.342448
1,XS1449458915,GOTRL 5.950 07/19/21,0.558,350000000.0,USD,10.0,5,2,2020-12-01,3.0,1,0.435783
2,XS1319822752,EVRE 8.250 01/28/21,0.097,750000000.0,USD,10.0,6,1,2020-12-01,-1.0,0,0.000000
3,XS1433454243,FASPM 5.375 06/16/23,2.358,900000000.0,USD,10.0,5,2,2020-12-01,3.0,0,0.428966
4,XS1951067039,CBOMF 5.150 02/20/24,2.890,327228000.0,EUR,10.0,4,2,2020-12-01,3.0,0,0.439385


In [ ]:
dummies = pd.get_dummies(combined_data['Currency'])
combined_data = pd.concat([combined_data, dummies], axis=1)
combined_data.drop(columns=['Currency'], inplace=True)

# Можно просто прочитать датасет с этого момента

Было принято решение убрать столбцы с валютой и ломбардный список

In [2]:
# combined_data.to_csv('echelons.csv', index=False)
combined_data = pd.read_csv('echelons.csv')
combined_data["Date"] = pd.to_datetime(combined_data["Date"])
combined_data.head()

,Index,Description,Duration,Amount,TRPS,Rating,Echelon,Date,Listing,Lombard,Liquidity,EUR,RUB,USD
0,XS2134628069,MMCFN 2.550 09/11/25 '25,4.223,500000000.0,10.0,7,1,2020-12-01,3.0,0,0.342448,0,0,1
1,XS1449458915,GOTRL 5.950 07/19/21,0.558,350000000.0,10.0,5,2,2020-12-01,3.0,1,0.435783,0,0,1
2,XS1319822752,EVRE 8.250 01/28/21,0.097,750000000.0,10.0,6,1,2020-12-01,-1.0,0,0.000000,0,0,1
3,XS1433454243,FASPM 5.375 06/16/23,2.358,900000000.0,10.0,5,2,2020-12-01,3.0,0,0.428966,0,0,1
4,XS1951067039,CBOMF 5.150 02/20/24,2.890,327228000.0,10.0,4,2,2020-12-01,3.0,0,0.439385,1,0,0


<b> Листинг -1 заменяем на 4, чтобы была лучше интерпретируемость </b>

In [3]:
combined_data.loc[combined_data['Listing'] == -1, 'Listing'] = 4

In [4]:
set(combined_data['Listing'])

{1.0, 2.0, 3.0, 4.0}

In [5]:
combined_data.loc[(combined_data['Index']=='RU000A100V79')]

,Index,Description,Duration,Amount,TRPS,Rating,Echelon,Date,Listing,Lombard,Liquidity,EUR,RUB,USD
32,RU000A100V79,TCSX 8.250 09/12/29 PUT,6.311,1.000000e+10,7.0,4,2,2020-12-01,1.0,0,0.511422,0,1,0
85,RU000A100V79,TCSX 8.250 09/12/29 PUT,6.221,1.000000e+10,8.0,4,2,2021-01-01,1.0,0,0.513494,0,1,0
143,RU000A100V79,TCSX 8.250 09/12/29 PUT,6.153,1.000000e+10,8.0,4,2,2021-02-01,1.0,0,0.512606,0,1,0
203,RU000A100V79,TCSX 8.250 09/12/29 PUT,6.046,1.000000e+10,7.0,4,2,2021-03-01,1.0,0,0.491023,0,1,0
266,RU000A100V79,TCSX 8.250 09/12/29 PUT,6.197,1.000000e+10,7.0,4,2,2021-04-01,1.0,0,0.519726,0,1,0
330,RU000A100V79,TCSX 8.250 09/12/29 PUT,6.132,1.000000e+10,7.0,4,2,2021-05-01,1.0,0,0.512483,0,1,0
396,RU000A100V79,TCSX 8.250 09/12/29 PUT,6.038,1.000000e+10,8.0,4,2,2021-06-01,1.0,0,0.507017,0,1,0
464,RU000A100V79,TCSX 8.250 09/12/29 PUT,5.947,1.000000e+10,8.0,4,2,2021-07-01,1.0,0,0.563355,0,1,0
530,RU000A100V79,TCSX 8.250 09/12/29 PUT,5.859,1.000000e+10,8.0,4,2,2021-08-01,1.0,0,0.585213,0,1,0
591,RU000A100V79,TCSX 8.250 09/12/29 PUT,5.780,1.000000e+10,8.0,4,2,2021-09-01,1.0,0,0.607570,0,1,0


3 2 3

In [6]:
combined_data.loc[(combined_data['Rating'] < 5.5)&(combined_data['Rating'] > 3.5)].sort_values('Echelon')

,Index,Description,Duration,Amount,TRPS,Rating,Echelon,Date,Listing,Lombard,Liquidity,EUR,RUB,USD
1,XS1449458915,GOTRL 5.950 07/19/21,0.558,3.500000e+08,10.0,5,2,2020-12-01,3.0,1,0.435783,0,0,1
524,XS2099763075,CBOMF 4.700 01/29/25,3.216,5.410000e+08,10.0,5,2,2021-08-01,3.0,0,0.526798,0,0,1
528,RU000A0JVUK8,AFKS 9.750 09/30/25 PUT,0.612,5.000000e+09,8.0,5,2,2021-08-01,3.0,1,0.419462,0,1,0
529,RU000A0JWYQ5,AFKS 6.350 10/30/26 PUT,4.418,6.500000e+09,9.0,5,2,2021-08-01,3.0,1,0.393671,0,1,0
530,RU000A100V79,TCSX 8.250 09/12/29 PUT,5.859,1.000000e+10,8.0,4,2,2021-08-01,1.0,0,0.585213,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,RU000A0JS603,VNESH 6.250 08/11/22 PUT,1.108,5.000000e+09,6.0,5,3,2021-06-01,2.0,1,0.000000,0,1,0
363,RU000A0JS603,VNESH 6.250 08/11/22 PUT,1.191,5.000000e+09,7.0,5,3,2021-05-01,2.0,1,0.000000,0,1,0
362,RU000A0JS5R1,VNESH 6.200 03/17/22 PUT,0.818,5.000000e+09,7.0,5,3,2021-05-01,2.0,1,0.000000,0,1,0
496,RU000A0JS603,VNESH 6.250 08/11/22 PUT,1.018,5.000000e+09,6.0,5,3,2021-07-01,2.0,1,0.000000,0,1,0


3 2 2

In [7]:
combined_data.loc[(combined_data['Rating'] < 5.5)&(combined_data['Rating'] > 3.5)&(combined_data['Liquidity'] < 0.532)&(combined_data['Echelon'] == 3)].sort_values('Index')

,Index,Description,Duration,Amount,TRPS,Rating,Echelon,Date,Listing,Lombard,Liquidity,EUR,RUB,USD
362,RU000A0JS5R1,VNESH 6.200 03/17/22 PUT,0.818,5.000000e+09,7.0,5,3,2021-05-01,2.0,1,0.000000,0,1,0
429,RU000A0JS5R1,VNESH 6.200 03/17/22 PUT,0.736,5.000000e+09,6.0,5,3,2021-06-01,2.0,1,0.000000,0,1,0
495,RU000A0JS5R1,VNESH 6.200 03/17/22 PUT,0.645,5.000000e+09,6.0,5,3,2021-07-01,2.0,1,0.000000,0,1,0
363,RU000A0JS603,VNESH 6.250 08/11/22 PUT,1.191,5.000000e+09,7.0,5,3,2021-05-01,2.0,1,0.000000,0,1,0
430,RU000A0JS603,VNESH 6.250 08/11/22 PUT,1.108,5.000000e+09,6.0,5,3,2021-06-01,2.0,1,0.000000,0,1,0
496,RU000A0JS603,VNESH 6.250 08/11/22 PUT,1.018,5.000000e+09,6.0,5,3,2021-07-01,2.0,1,0.000000,0,1,0
339,RU000A102GY7,HEDTR 6.450 12/11/23,2.370,4.000000e+09,6.0,5,3,2021-05-01,3.0,0,0.515653,0,1,0


In [8]:
combined_data.loc[(combined_data['Rating'] > 3.5)&(combined_data['Rating'] > 5.5)&(combined_data['Rating'] < 9.5)&(combined_data['Amount'] > 3500000000)&(combined_data['Listing'] >1.5)&(combined_data['Amount'] <= 8500000000)&(combined_data['Duration'] <=1.965)&(combined_data['Rating'] > 6.5)&(combined_data['Duration'] <= 1.126)&(combined_data['Amount'] <= 6000000000)].sort_values('Index')

,Index,Description,Duration,Amount,TRPS,Rating,Echelon,Date,Listing,Lombard,Liquidity,EUR,RUB,USD
28,RU000A0JREW5,VNESH 7.270 04/02/21 PUT,0.277,5.000000e+09,8.0,7,1,2020-12-01,3.0,1,0.247987,0,1,0
81,RU000A0JREW5,VNESH 7.270 04/02/21 PUT,0.192,5.000000e+09,8.0,7,1,2021-01-01,3.0,1,0.227047,0,1,0
139,RU000A0JREW5,VNESH 7.270 04/02/21 PUT,0.000,5.000000e+09,8.0,7,1,2021-02-01,3.0,1,0.000000,0,1,0
199,RU000A0JREW5,VNESH 7.270 04/02/21 PUT,0.041,5.000000e+09,8.0,7,1,2021-03-01,3.0,1,0.152813,0,1,0
743,RU000A0JTLJ3,USMHD 7.650 01/27/23 PUT,1.123,5.000000e+09,9.0,7,1,2021-11-01,2.0,1,0.182875,0,1,0
807,RU000A0JTLJ3,USMHD 7.650 01/27/23 PUT,1.057,5.000000e+09,8.0,7,1,2021-12-01,2.0,1,0.248516,0,1,0
873,RU000A0JTLJ3,USMHD 7.650 01/27/23 PUT,0.961,5.000000e+09,8.0,7,1,2022-01-01,2.0,1,0.353016,0,1,0
744,RU000A0JTLL9,USMHD 7.650 01/27/23 PUT,1.123,5.000000e+09,8.0,7,1,2021-11-01,2.0,1,0.271899,0,1,0
808,RU000A0JTLL9,USMHD 7.650 01/27/23 PUT,1.057,5.000000e+09,8.0,7,1,2021-12-01,2.0,1,0.248259,0,1,0
874,RU000A0JTLL9,USMHD 7.650 01/27/23 PUT,0.961,5.000000e+09,7.0,7,2,2022-01-01,2.0,1,0.240876,0,1,0


Убрать: RU000A0JVVA7 RU000A1008V9 RU000A101KK0
RU000A0JS5R1 RU000A0JS603 
RU000A101MZ4

In [9]:
remove_indexes = ['RU000A0JVVA7', 'RU000A1008V9', 'RU000A101KK0', 'RU000A0JS5R1', 'RU000A0JS603', 'RU000A101MZ4']
combined_data = combined_data[~combined_data['Index'].isin(remove_indexes)]

Удаление облигаций с 0 ликвидностью

In [10]:
# df_delete = combined_data[(combined_data['Liquidity']==0)&((combined_data['EUR']==1)|(combined_data['USD']==1))]
df_delete = combined_data[(combined_data['Liquidity']==0)]
# Получаем индексы строк, которые нужно удалить
indices_to_delete = df_delete.index
# Удаляем строки из combined_data
combined_data = combined_data.drop(indices_to_delete)
combined_data = combined_data.reset_index(drop=True)
combined_data.head()

,Index,Description,Duration,Amount,TRPS,Rating,Echelon,Date,Listing,Lombard,Liquidity,EUR,RUB,USD
0,XS2134628069,MMCFN 2.550 09/11/25 '25,4.223,500000000.0,10.0,7,1,2020-12-01,3.0,0,0.342448,0,0,1
1,XS1449458915,GOTRL 5.950 07/19/21,0.558,350000000.0,10.0,5,2,2020-12-01,3.0,1,0.435783,0,0,1
2,XS1433454243,FASPM 5.375 06/16/23,2.358,900000000.0,10.0,5,2,2020-12-01,3.0,0,0.428966,0,0,1
3,XS1951067039,CBOMF 5.150 02/20/24,2.890,327228000.0,10.0,4,2,2020-12-01,3.0,0,0.439385,1,0,0
4,XS2010044381,GTEPT 4.949 02/18/26,4.555,550000000.0,10.0,5,2,2020-12-01,3.0,1,0.499631,0,0,1


In [11]:
df_delete.to_excel('df_delete.xlsx')

Удаление ЕВРО и ДОЛЛАР бумаг

In [12]:
df_delete = combined_data[(combined_data['EUR']==1) | (combined_data['USD']==1)]
# Получаем индексы строк, которые нужно удалить
indices_to_delete = df_delete.index
# Удаляем строки из combined_data
combined_data = combined_data.drop(indices_to_delete)
combined_data = combined_data.reset_index(drop=True)
combined_data.head()

,Index,Description,Duration,Amount,TRPS,Rating,Echelon,Date,Listing,Lombard,Liquidity,EUR,RUB,USD
0,RU000A0JREW5,VNESH 7.270 04/02/21 PUT,0.277,5.000000e+09,8.0,7,1,2020-12-01,3.0,1,0.247987,0,1,0
1,RU000A0JWV63,FASPM 14.250 Perp '26,4.214,5.000000e+09,7.0,6,2,2020-12-01,3.0,0,0.506778,0,1,0
2,RU000A0JVUK8,AFKS 9.750 09/30/25 PUT,1.226,5.000000e+09,9.0,4,2,2020-12-01,3.0,1,0.479351,0,1,0
3,RU000A0JWYQ5,AFKS 6.350 10/30/26 PUT,4.943,6.500000e+09,9.0,4,2,2020-12-01,3.0,1,0.609514,0,1,0
4,RU000A100V79,TCSX 8.250 09/12/29 PUT,6.311,1.000000e+10,7.0,4,2,2020-12-01,1.0,0,0.511422,0,1,0


In [18]:
combined_data.loc[(combined_data['Rating'] > 5.5)&(combined_data['Listing'] < 1.5)&(combined_data['Duration'] >= 2.165)].sort_values('Index')

,Index,Description,Duration,Amount,TRPS,Rating,Echelon,Date,Listing,Lombard,Liquidity,EUR,RUB,USD
18,RU000A0JVA10,GOTRL 7.980 03/06/25 PUT,2.471,5.000000e+09,8.0,6,1,2020-12-01,1.0,1,0.446915,0,1,0
43,RU000A0JVA10,GOTRL 7.980 03/06/25 PUT,2.381,5.000000e+09,8.0,6,1,2021-01-01,1.0,1,0.406625,0,1,0
129,RU000A0JVA10,GOTRL 7.980 03/06/25 PUT,2.182,5.000000e+09,8.0,6,1,2021-04-01,1.0,1,0.379386,0,1,0
70,RU000A0JVA10,GOTRL 7.980 03/06/25 PUT,2.319,5.000000e+09,8.0,6,1,2021-02-01,1.0,1,0.413961,0,1,0
101,RU000A0JVA10,GOTRL 7.980 03/06/25 PUT,2.278,5.000000e+09,8.0,6,1,2021-03-01,1.0,1,0.405116,0,1,0
14,RU000A0ZYNY4,GOTRL 5.250 12/31/32 FRN PUT,8.706,1.000000e+10,8.0,6,1,2020-12-01,1.0,1,0.487284,0,1,0
267,RU000A0ZYNY4,GTLK 6.250 12/31/32 FRN PUT,7.443,1.000000e+10,8.0,6,1,2021-09-01,1.0,1,0.446244,0,1,0
241,RU000A0ZYNY4,GOTRL 6.250 12/31/32 FRN PUT,7.639,1.000000e+10,8.0,6,1,2021-08-01,1.0,1,0.476362,0,1,0
323,RU000A0ZYNY4,GTLK 6.250 12/31/32 FRN PUT,7.255,1.000000e+10,8.0,6,1,2021-11-01,1.0,1,0.475264,0,1,0
213,RU000A0ZYNY4,GOTRL 6.250 12/31/32 FRN PUT,8.068,1.000000e+10,8.0,6,1,2021-07-01,1.0,1,0.493651,0,1,0


In [14]:
combined_data.loc[(combined_data['Rating'] > 5.5)&(combined_data['Listing'] > 1.5)&(combined_data['Duration'] <= 2.012)&(combined_data['Liquidity'] > 0.242)&(combined_data['Duration'] > 1.115)&(combined_data['Amount'] <= 12500000256)&(combined_data['Index'] == 'RU000A0JTLL9')].sort_values('Date').reset_index()

,index,Index,Description,Duration,Amount,TRPS,Rating,Echelon,Date,Listing,Lombard,Liquidity,EUR,RUB,USD
0,23,RU000A0JTLL9,USMHD 7.650 01/27/23 PUT,1.924,5.000000e+09,8.0,6,1,2020-12-01,2.0,1,0.412047,0,1,0
1,50,RU000A0JTLL9,USMHD 7.650 01/27/23 PUT,1.831,5.000000e+09,7.0,6,2,2021-01-01,2.0,1,0.465442,0,1,0
2,78,RU000A0JTLL9,USMHD 7.650 01/27/23 PUT,1.839,5.000000e+09,8.0,6,1,2021-02-01,2.0,1,0.469726,0,1,0
3,109,RU000A0JTLL9,USMHD 7.650 01/27/23 PUT,1.755,5.000000e+09,7.0,6,2,2021-03-01,2.0,1,0.263076,0,1,0
4,137,RU000A0JTLL9,USMHD 7.650 01/27/23 PUT,1.658,5.000000e+09,7.0,6,2,2021-04-01,2.0,1,0.386219,0,1,0
5,166,RU000A0JTLL9,USMHD 7.650 01/27/23 PUT,1.589,5.000000e+09,7.0,6,2,2021-05-01,2.0,1,0.450325,0,1,0
6,194,RU000A0JTLL9,USMHD 7.650 01/27/23 PUT,1.506,5.000000e+09,8.0,6,1,2021-06-01,2.0,1,0.263790,0,1,0
7,224,RU000A0JTLL9,USMHD 7.650 01/27/23 PUT,1.416,5.000000e+09,8.0,6,1,2021-07-01,2.0,1,0.245140,0,1,0
8,251,RU000A0JTLL9,USMHD 7.650 01/27/23 PUT,1.384,5.000000e+09,7.0,6,2,2021-08-01,2.0,1,0.426755,0,1,0
9,305,RU000A0JTLL9,USMHD 7.650 01/27/23 PUT,1.214,5.000000e+09,7.0,7,2,2021-10-01,2.0,1,0.298697,0,1,0


In [13]:
y = combined_data['Echelon'].values

In [14]:
# Сколько элементов в каждом классе
for i in range(1, 4):
    print(np.count_nonzero(y == i))

191
180
20


<b>Мало элементов из 3 класса, сделаем оверсемплинг</b> 

In [15]:
from imblearn.over_sampling import SMOTE

In [16]:
## Делаем оверсемплинг только трейн данных
# Определение категориальных признаков (дискретных)
discrete_cols = ['Rating', 'Listing']

# Определение непрерывных признаков
continuous_cols = ['Duration', 'Amount', 'Liquidity']

# Разделение признаков X и целевой переменной y
X = combined_data[['Duration', 'Amount', 'Rating', 'Listing', 'Liquidity']]
y = combined_data['Echelon']

# Определение индексов категориальных признаков для SMOTENC
categorical_feature_indices = [X.columns.get_loc(col) for col in discrete_cols]

# Применение SMOTENC для оверсемплинга как непрерывных, так и категориальных признаков
smotenc = SMOTENC(categorical_features=categorical_feature_indices, random_state=42)
X_resampled, y_resampled = smotenc.fit_resample(X, y)

X_resampled = X_resampled.values
y_resampled = y_resampled.values

In [17]:
X_resampled

array([[2.77000000e-01, 5.00000000e+09, 7.00000000e+00, 3.00000000e+00,
        2.47987315e-01],
       [4.21400000e+00, 5.00000000e+09, 6.00000000e+00, 3.00000000e+00,
        5.06778419e-01],
       [1.22600000e+00, 5.00000000e+09, 4.00000000e+00, 3.00000000e+00,
        4.79351197e-01],
       ...,
       [2.32000000e+00, 6.00000000e+09, 3.00000000e+00, 3.00000000e+00,
        5.58611781e-01],
       [2.25920868e+00, 4.79356765e+09, 3.00000000e+00, 3.00000000e+00,
        5.18303502e-01],
       [4.69373495e+00, 1.00000000e+10, 3.00000000e+00, 2.00000000e+00,
        7.08158318e-01]])

In [18]:
np.count_nonzero(y_resampled == 1)

191

In [19]:
# Сколько элементов в каждом классе
for i in range(1, 4):
    print(np.count_nonzero(y_resampled == i))

191
191
191


In [20]:
# 'Duration', 'Amount', 'Rating', 'Listing', 'Liquidity'
for depth in range(1, 15):
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=30)
    clf = DecisionTreeClassifier(random_state=18, max_depth=depth)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy)
    feature_importances = clf.feature_importances_
    print('Значимость признаков:')
    print(feature_importances)

0.6173913043478261
Значимость признаков:
[0. 0. 1. 0. 0.]
0.8608695652173913
Значимость признаков:
[0.01973396 0.         0.98026604 0.         0.        ]
0.8695652173913043
Значимость признаков:
[0.02032169 0.00847929 0.90295029 0.05983656 0.00841217]
0.9304347826086956
Значимость признаков:
[0.10969072 0.0076623  0.81903647 0.05407126 0.00953925]
0.9304347826086956
Значимость признаков:
[0.12335505 0.00716693 0.76608493 0.05057549 0.0528176 ]
0.9304347826086956
Значимость признаков:
[0.13752263 0.00694329 0.75536734 0.04899731 0.05116944]
0.9304347826086956
Значимость признаков:
[0.13497765 0.01924811 0.74138863 0.04809057 0.05629503]
0.9391304347826087
Значимость признаков:
[0.13297384 0.01896237 0.73038234 0.04737664 0.07030481]
0.9217391304347826
Значимость признаков:
[0.13797531 0.01874288 0.72192835 0.04682827 0.0745252 ]
0.9217391304347826
Значимость признаков:
[0.13706407 0.0186191  0.71716047 0.046519   0.08063737]
0.9217391304347826
Значимость признаков:
[0.13662238 0.01855

## Обучение окончено, глубина дерева = не ограничена, вручную отсечем ненужные ветви

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=30)
clf = DecisionTreeClassifier(random_state=18)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)
feature_importances = clf.feature_importances_
print('Значимость признаков:')
print(feature_importances)

0.9217391304347826
Значимость признаков:
[0.14367765 0.02210552 0.70463144 0.04570629 0.08387909]


In [30]:
cols = ['Duration', 'Amount', 'Rating', 'Listing', 'Liquidity']
test_df = pd.DataFrame(X_test, columns=cols)
test_df['y_real'] = y_test

In [32]:
test_df.to_excel('test_df.xlsx')

In [46]:
features = ['Duration', 'Amount', 'Rating', 'Listing', 'Liquidity']

In [47]:
import graphviz
# # Обучение модели
# clf = DecisionTreeClassifier(random_state=42, max_depth=7)
# clf.fit(df_train_important, y_train)

# Экспортируем дерево в формат dot
dot_data = export_graphviz(clf, out_file=None, 
                           feature_names=features,   
                           class_names=['Эшелон 1', 'Эшелон 2', 'Эшелон 3'], 
                           filled=True, rounded=True,  
                           special_characters=True)

# Генерируем изображение дерева
graph = graphviz.Source(dot_data)
graph.render("Убраны евро")  # Сохранение в файл tree.pdf

# Визуализация
graph.view()

'Убраны евро.pdf'

Затест на определенной дате

In [15]:
X = combined_data[combined_data['Date'] == '2022-01-01'][['Duration', 'Amount', 'Rating', 'Listing', 'Liquidity']].values
y = combined_data[combined_data['Date'] == '2022-01-01']['Echelon'].values

In [16]:
y_pred = clf.predict(X)
accuracy = accuracy_score(y, y_pred)
print(accuracy)

1.0


In [17]:
for i in range(len(y)):
    print(y[i], y_pred[i])

2 2
1 1
1 1
1 1
1 1
2 2
1 1
2 2
2 2
2 2
1 1
1 1
1 1
1 1
1 1
2 2
1 1
2 2
2 2
1 1
2 2
2 2
2 2
2 2
1 1
2 2
1 1
1 1
2 2
2 2
2 2
3 3
2 2
1 1
1 1
1 1
1 1
1 1
1 1
1 1
2 2
2 2
1 1
3 3
2 2
1 1
2 2
3 3
2 2
3 3
1 1


In [18]:
combined_data[combined_data['Date'] == '2022-01-01']

,Index,Description,Duration,Amount,TRPS,Rating,Echelon,Date,Listing,Lombard,Liquidity,EUR,RUB,USD
624,RU000A101616,FASPM 5.000 Perp '25,19.322,1.500000e+08,7.0,6,2,2022-01-01,3.0,0,0.456713,1,0,0
625,XS2159874002,LKOH 3.875 05/06/30,7.089,1.500000e+09,10.0,8,1,2022-01-01,3.0,0,0.524221,0,0,1
626,XS2134628069,MMCFN 2.550 09/11/25 '25,3.461,5.000000e+08,10.0,7,1,2022-01-01,3.0,0,0.460428,0,0,1
627,XS2291819980,GAZFI 2.950 01/27/29 MTN,6.252,2.000000e+09,10.0,8,1,2022-01-01,3.0,0,0.482264,0,0,1
628,XS0810596832,VTBR 9.500 Perp '22 FRN,0.853,2.250000e+09,10.0,7,1,2022-01-01,3.0,0,0.636351,0,0,1
629,XS2384475930,CBOMF 3.875 09/21/26 '26,4.224,5.000000e+08,9.0,5,2,2022-01-01,3.0,0,0.554729,0,0,1
630,XS1433454243,FLOT 5.375 06/16/23,1.365,4.984270e+08,10.0,7,1,2022-01-01,3.0,0,0.303208,0,0,1
631,XS1772800204,DMAIR 5.075 02/15/23 '22,0.792,2.047190e+08,10.0,5,2,2022-01-01,3.0,0,0.346649,0,0,1
632,XS2010044381,GTEPT 4.949 02/18/26,3.671,5.500000e+08,10.0,5,2,2022-01-01,3.0,1,0.460720,0,0,1
633,XS2063279959,ALFAB 5.950 04/15/30 '25 FRN,2.940,8.500000e+08,10.0,5,2,2022-01-01,3.0,0,0.518744,0,0,1


# Запросы по API

In [2]:
import requests
import xml.etree.ElementTree as ET

In [3]:
# Достаем isin облигаций из портфеля
df_isin = pd.read_excel("sec_011024.xlsx")

In [7]:
isin_list = df_isin['ISIN'].tolist()
# Букавы рейтинг меняем на машина понимать язык
replacement_dict = {
    'NR': 0,
    'B-': 1,
    'B': 2,
    'B+': 3,
    'BB-': 4,
    'BB': 5,
    'BB+': 6,
    'BBB-': 7,
    'BBB': 8,
    'BBB+': 9,
    'A-': 10
}
df_isin['Рейтинг'] = df_isin['Рейтинг'].replace(replacement_dict)

In [8]:
set(df_isin['Currency'])

{'AED', 'CNY', 'EUR', 'RUB', 'USD'}

In [10]:
df_isin

,ISIN,Duration,Currency,Рейтинг,Amount Outstanding
0,RU000A102CL3,6.676644,EUR,8,1250000000
1,RU000A102CL3,6.676644,EUR,8,1250000000
2,RU000A106V57,0.874183,AED,5,370000000
3,RU000A1080Z9,1.160000,CNY,3,160289000
4,RU000A1082Q4,0.015631,USD,7,1676729000
...,...,...,...,...,...
110,RU000A107DS5,0.500000,RUB,7,5500000000
111,RU000A107EA1,0.500000,RUB,7,10000000000
112,RU000A107UX9,0.246575,RUB,7,20000000000
113,RU000A108P46,0.082192,RUB,5,6000000000


In [11]:
isin_list

['RU000A102CL3',
 'RU000A102CL3',
 'RU000A106V57',
 'RU000A1080Z9',
 'RU000A1082Q4',
 'RU000A108Q29',
 'XS2120091991',
 'RU000A102K96',
 'RU000A105KU0',
 'RU000A107AQ5',
 'RU000A107B43',
 'RU000A107CX7',
 'RU000A1059R0',
 'RU000A105M75',
 'RU000A1086M4',
 'RU000A107VV1',
 'XS2281299763',
 'RU000A101616',
 'RU000A106V57',
 'XS2010040397',
 'RU000A1086Z6',
 'RU000A107X96',
 'RU000A107XA1',
 'RU000A107AU7',
 'RU000A106LL5',
 'RU000A108KT6',
 'RU000A109KT4',
 'RU000A108P46',
 'RU000A106EM8',
 'RU000A105WC3',
 'RU000A105XE7',
 'RU000A105X80',
 'RU000A107605',
 'RU000A107HH9',
 'RU000A107UX9',
 'RU000A109B33',
 'RU000A107050',
 'RU000A107084',
 'RU000A107ET1',
 'RU000A108JV4',
 'RU000A0ZYR91',
 'RU000A105KW6',
 'RU000A1064L3',
 'RU000A107EA1',
 'RU000A106F40',
 'RU000A108Y86',
 'RU000A107DS5',
 'RU000A108Q03',
 'RU000A1083W0',
 'RU000A1078S8',
 'RU000A107456',
 'RU000A1083A6',
 'RU000A109DT9',
 'RU000A101780',
 'RU000A101T56',
 'RU000A101UE2',
 'RU000A1077V4',
 'RU000A108ZT4',
 'RU000A107RZ0

In [12]:
def currency_exc(date_from, date_till):
    data = []
    
    # Словарь с кодами валют
    currency_codes = {
        'USD': 'R01235',
        'EUR': 'R01239',
        'AED': 'R01230',
        'CNY': 'R01375'
        # Добавьте другие валюты по необходимости
    }
    
    # Преобразование строк с датами в формат 'dd/mm/yyyy'
    date_from_str = datetime.strptime(date_from, '%Y-%m-%d').strftime('%d/%m/%Y')
    date_till_str = datetime.strptime(date_till, '%Y-%m-%d').strftime('%d/%m/%Y')
    
    for char_code, iso_code in currency_codes.items():
        # Формирование URL для запроса
        url = f'https://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={date_from_str}&date_req2={date_till_str}&VAL_NM_RQ={iso_code}'

        # Запрос данных
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Ошибка при запросе данных для {char_code}: {response.status_code}")
            continue

        # Парсинг XML-ответа
        root = ET.fromstring(response.content)
        for record in root.findall('Record'):
            date = record.attrib['Date']
            value = record.find('Value').text
            nominal = int(record.find('Nominal').text)
            rate = float(value.replace(',', '.')) / nominal
            data.append({
                'Date': datetime.strptime(date, '%d.%m.%Y').date(),
                'Currency': char_code,
                'Rate': rate
            })

    # Создание DataFrame
    df = pd.DataFrame(data)

    # Заполнение пропущенных дат
    date_from_obj = datetime.strptime(date_from, '%Y-%m-%d').date()
    date_till_obj = datetime.strptime(date_till, '%Y-%m-%d').date()
    all_dates = pd.date_range(start=date_from_obj, end=date_till_obj)

    full_df = pd.DataFrame()

    for currency in currency_codes.keys():
        currency_df = df[df['Currency'] == currency]
        currency_df = currency_df.set_index('Date').reindex(all_dates).fillna(method='ffill').reset_index()
        currency_df['Currency'] = currency
        full_df = pd.concat([full_df, currency_df])

    # Переименование столбцов после объединения
    full_df.columns = ['Date', 'Currency', 'Rate']
    full_df = full_df[['Date', 'Currency', 'Rate']]

    # Преобразование столбца 'Date' в тип datetime.date
    full_df['Date'] = full_df['Date'].apply(lambda x: x.date())

    return full_df

In [13]:
# Создаем датафрейм курсов валют по датам НУЖНО 30 ДНЕЙ
date_from = '2024-08-20'
date_till = '2024-09-30'
df_currency = currency_exc(date_from, date_till)

In [14]:
df_currency.head()

,Date,Currency,Rate
0,2024-08-20,USD,89.5428
1,2024-08-21,USD,90.6944
2,2024-08-22,USD,91.6862
3,2024-08-23,USD,91.2881
4,2024-08-24,USD,91.6012


## Подсчет среднего объема и кол-ва сделок для каждой бумаги

In [15]:
value_list = []
num_trades_list = []
listing_list = []

for isin in isin_list:
    # isin = 'RU000A102CL3'
    print(isin)
    url_isin = f'https://iss.moex.com/iss/securities?q={isin}&securities.columns=secid'
    # Отправка GET-запроса чтобы вытащить secid
    response_isin = requests.get(url_isin)
    time.sleep(1)
    try:
        if response_isin.status_code == 200:
            data_secid = pd.read_xml(response_isin.content, xpath=".//rows/row")
            secid = data_secid.iloc[0]['secid']
            isin = secid
            print(isin)
        else:
            print(f"Ошибка при доступе к API MOEX: {url_isin}")
    except:
        print(f'Нет secid для isin')
    # URL для получения данных по облигациям, котируемым на MOEX
    url = f'https://iss.moex.com/iss/history/engines/stock/markets/bonds/securities/{isin}?from={date_from}&till={date_till}&sort_order=desc&iss.only=history'
    url_listing = f'https://iss.moex.com/iss/engines/stock/markets/bonds/securities/{isin}?securities.columns=SECID,LISTLEVEL&iss.only=securities&iss.only=history'
    # Отправка GET-запроса к API MOEX
    response = requests.get(url)
    time.sleep(1)
    # Переменные для сумм объемов
    value_sum = 0
    num_trades_sum = 0
    try:
        # Проверка статуса ответа
        if response.status_code == 200:
            # Парсинг XML
            df = pd.read_xml(response.content, xpath=".//rows/row") 
            df = df[['TRADEDATE', 'BOARDID', 'VALUE', 'VOLUME', 'WAPRICE', 'FACEUNIT', 'CURRENCYID', 'FACEVALUE', 'NUMTRADES']]

            # Подсчет количества торговых дней
            days = df['TRADEDATE'].nunique()

            # Удаляются все строки, в столбце 'VALUE' которых значения равны 0 либо Nan
            mask = ((df['VALUE'] == 0) | df['VALUE'].isna())
            df = df[~mask]

            # Пробегаемся по строкам, складываем VALUE*валюта расчета
            for index, row in df.iterrows():
                # курс валюты строки к рублю
                date = datetime.strptime(row['TRADEDATE'], '%Y-%m-%d').date()

                if row['CURRENCYID'] == 'SUR' or row['CURRENCYID'] == 'RUR' or row['CURRENCYID'] == 'RUB':
                    currency = 1
                else:
                    # ищем строку с курсом
                    result = df_currency.loc[(df_currency['Date'] == date) & (df_currency['Currency'] == row['CURRENCYID'])]
                    if not result.empty:
                        currency = result.iloc[0]['Rate']
#                         print(f"\nКурс {row['CURRENCYID']} на {date}: {currency}")
                    else:
                        print(f"\nЗапись не найдена {row['CURRENCYID']} на {date}")

                # умножаем объем на курс
                value_row = row['VALUE'] * currency
                value_sum += value_row
                num_trades_sum += row['NUMTRADES']

            value_list.append(value_sum/days)
            num_trades_list.append(num_trades_sum/days)
        else:
            print(f"Ошибка при доступе к API MOEX: {url}")
    except:
        value_list.append(0)
        num_trades_list.append(0)
        print(f"Нет данных торгов для ISIN: {isin}")

    # Отправка GET-запроса к API MOEX
    response_listing = requests.get(url_listing)
    time.sleep(1)

    try:
        # Проверка статуса ответа
        if response_listing.status_code == 200:
            # Парсинг XML
            data_listing = pd.read_xml(response_listing.content, xpath=".//rows/row")
            listing_rank = data_listing.iloc[0]['LISTLEVEL']
            listing_list.append(listing_rank)
            print(listing_rank)
        else:
            print(f"Ошибка при доступе к API MOEX: {url_listing}")
    except:
        listing_rank = 4
        listing_list.append(listing_rank)
        print(f"Нет данных листинга для ISIN: {isin}")

RU000A102CL3
RU000A102CL3
1.0
RU000A102CL3
RU000A102CL3
1.0
RU000A106V57
RU000A106V57
2.0
RU000A1080Z9
RU000A1080Z9
3.0
RU000A1082Q4
RU000A1082Q4
3.0
RU000A108Q29
RU000A108Q29
3.0
XS2120091991
XS2120091991
Нет данных торгов для ISIN: XS2120091991
Нет данных листинга для ISIN: XS2120091991
RU000A102K96
RU000A102K96
1.0
RU000A105KU0
RU000A105KU0
3.0
RU000A107AQ5
RU000A107AQ5
3.0
RU000A107B43
RU000A107B43
3.0
RU000A107CX7
RU000A107CX7
3.0
RU000A1059R0
RU000A1059R0
3.0
RU000A105M75
RU000A105M75
3.0
RU000A1086M4
RU000A1086M4
3.0
RU000A107VV1
RU000A107VV1
3.0
XS2281299763
XS2281299763
Нет данных торгов для ISIN: XS2281299763
Нет данных листинга для ISIN: XS2281299763
RU000A101616
RU000A101616
3.0
RU000A106V57
RU000A106V57
2.0
XS2010040397
XS2010040397
Нет данных листинга для ISIN: XS2010040397
RU000A1086Z6
RU000A1086Z6
Нет данных торгов для ISIN: RU000A1086Z6
Нет данных листинга для ISIN: RU000A1086Z6
RU000A107X96
RU000A107X96
3.0
RU000A107XA1
RU000A107XA1
3.0
RU000A107AU7
RU000A107AU7
3.0
R

In [16]:
df_isin['Value_avg'] = value_list
df_isin['Num_trades_avg'] = num_trades_list
df_isin['Listing'] = listing_list

In [17]:
df_isin.head()

,ISIN,Duration,Currency,Рейтинг,Amount Outstanding,Value_avg,Num_trades_avg,Listing
0,RU000A102CL3,6.676644,EUR,8,1250000000,2.882338e+06,0.333333,1.0
1,RU000A102CL3,6.676644,EUR,8,1250000000,2.882338e+06,0.333333,1.0
2,RU000A106V57,0.874183,AED,5,370000000,3.039607e+04,1.066667,2.0
3,RU000A1080Z9,1.160000,CNY,3,160289000,1.787892e+06,59.933333,3.0
4,RU000A1082Q4,0.015631,USD,7,1676729000,4.296764e+07,154.200000,3.0


## Подсчет среднедневного объема и кол-ва сделок по всем инструментам

In [18]:
# Создаем список дат между from и till
date_list = pd.date_range(start=date_from, end=date_till).strftime('%Y-%m-%d').tolist()
# Создаем список интересующих режимов торгов
boards_list = ['TQCB', 'TQOE', 'TQOD', 'TQOY']

In [34]:
# date = date_list[0]
# board = boards_list[2]
num_trades_all_list = []
value_all_list = []
for date in date_list:
    for board in boards_list:
        # Базовый URL для запроса
        base_url = f'https://iss.moex.com/iss/history/engines/stock/markets/bonds/boards/{board}/securities.xml'

        params = {
            'history.columns': 'SECID,NUMTRADES,VALUE,CURRENCYID',
            'iss.only': 'history',
            'date': date,
            'start': 0
        }

        # Читаем 1 дату
        val_sum_day = 0
        num_trades_day = 0
        while True:
            # Отправка GET-запроса к API MOEX
            response = requests.get(base_url, params=params)

            # Проверка статуса ответа
            if response.status_code == 200:
                try:
                    # Парсинг XML
                    df = pd.read_xml(response.content, xpath=".//rows/row")

                    if df.empty:
                        break  # Если данных больше нет, прекращаем запросы

                    # Удаляются все строки, в столбце 'VALUE' которых значения равны 0 либо Nan
                    mask = ((df['VALUE'] == 0) | df['VALUE'].isna())
                    df = df[~mask]

                    # Пробегаемся по строкам, складываем VALUE*валюта расчета
                    for index, row in df.iterrows():
                        # курс валюты строки к рублю
                        date_cur = datetime.strptime(date, '%Y-%m-%d').date()

                        if row['CURRENCYID'] == 'SUR' or row['CURRENCYID'] == 'RUR' or row['CURRENCYID'] == 'RUB':
                            currency = 1
                        else:
                            # ищем строку с курсом
                            result = df_currency.loc[(df_currency['Date'] == date_cur) & (df_currency['Currency'] == row['CURRENCYID'])]
                            if not result.empty:
                                currency = result.iloc[0]['Rate']
                            else:
                                print(f"\nЗапись не найдена {row['CURRENCYID']} на {date_cur}")

                        # умножаем объем на курс
                        value_row = row['VALUE'] * currency
                        val_sum_day += value_row

                    num_trades_day += sum(df['NUMTRADES'])

                    # Увеличиваем смещение для следующего запроса
                    params['start'] += 100

                    # Задержка для предотвращения слишком частых запросов
                    # time.sleep(2)
                except:
                    if params['start'] > 0:
                        print('Парсинг закончен')
                    else: 
                        print(f'Нет никаких данных по ссылке: {base_url}, {params}')
                    break
            else:
                print(f"Ошибка при доступе к API MOEX: {base_url}, {params}")
                break

        value_all_list.append(val_sum_day)
        num_trades_all_list.append(num_trades_day)

Парсинг закончен
Парсинг закончен
Парсинг закончен
Парсинг закончен
Парсинг закончен
Парсинг закончен
Парсинг закончен
Парсинг закончен
Парсинг закончен
Парсинг закончен
Парсинг закончен
Парсинг закончен
Парсинг закончен
Парсинг закончен
Парсинг закончен
Парсинг закончен
Нет никаких данных по ссылке: https://iss.moex.com/iss/history/engines/stock/markets/bonds/boards/TQCB/securities.xml, {'history.columns': 'SECID,NUMTRADES,VALUE,CURRENCYID', 'iss.only': 'history', 'date': '2024-08-24', 'start': 0}
Нет никаких данных по ссылке: https://iss.moex.com/iss/history/engines/stock/markets/bonds/boards/TQOE/securities.xml, {'history.columns': 'SECID,NUMTRADES,VALUE,CURRENCYID', 'iss.only': 'history', 'date': '2024-08-24', 'start': 0}
Нет никаких данных по ссылке: https://iss.moex.com/iss/history/engines/stock/markets/bonds/boards/TQOD/securities.xml, {'history.columns': 'SECID,NUMTRADES,VALUE,CURRENCYID', 'iss.only': 'history', 'date': '2024-08-24', 'start': 0}
Нет никаких данных по ссылке: ht

### Итак, среднедневной объем и кол-ва сделок по всем инструментам равно:

In [36]:
# Cреднедневной объем
value_all_avg = sum(value_all_list)/30
# Среднедневное кол-во сделок
num_trades_all_avg = sum(num_trades_all_list)/30
print(f'Cреднедневной объем c {date_from} по {date_till}: {value_all_avg}')
print(f'Cреднедневное кол-во сделок c {date_from} по {date_till}: {num_trades_all_avg}')

Cреднедневной объем c 2024-08-20 по 2024-09-30: 8259855787.707852
Cреднедневное кол-во сделок c 2024-08-20 по 2024-09-30: 114427.76666666666


In [37]:
df_isin.head(3)

,ISIN,Duration,Currency,Рейтинг,Amount Outstanding,Value_avg,Num_trades_avg,Listing
0,RU000A102CL3,6.676644,EUR,8,1250000000,2.882338e+06,0.333333,1.0
1,RU000A102CL3,6.676644,EUR,8,1250000000,2.882338e+06,0.333333,1.0
2,RU000A106V57,0.874183,AED,5,370000000,3.039607e+04,1.066667,2.0


## Рассчитываем ликвидность

In [38]:
liquidity_list = []
# Коэффициенты из формулы
a = 0.03
b = 0.07
for index, row in df_isin.iterrows():
    liquidity_row = ((row['Value_avg']/value_all_avg)**a) * ((row['Num_trades_avg']/num_trades_all_avg)**b)
    liquidity_list.append(liquidity_row)

In [39]:
df_final = pd.DataFrame()
df_final['ISIN'] = df_isin['ISIN']
df_final['Duration'] = df_isin['Duration'] 
df_final['Amount'] = df_isin['Amount Outstanding'] 
df_final['Rating'] = df_isin['Рейтинг'] 
df_final['Listing'] = df_isin['Listing'] 
df_final['Liquidity'] = liquidity_list 
df_final['Currency'] = df_isin['Currency']

In [40]:
df_final.head()

,ISIN,Duration,Amount,Rating,Listing,Liquidity,Currency
0,RU000A102CL3,6.676644,1250000000,8,1.0,0.322691,EUR
1,RU000A102CL3,6.676644,1250000000,8,1.0,0.322691,EUR
2,RU000A106V57,0.874183,370000000,5,2.0,0.305380,AED
3,RU000A1080Z9,1.160000,160289000,3,3.0,0.457510,CNY
4,RU000A1082Q4,0.015631,1676729000,7,3.0,0.537717,USD


In [42]:
df_final.to_excel('sec_dop_011024.xlsx')

In [41]:
X = df_final[['Duration', 'Amount', 'Rating', 'Listing', 'Liquidity']].values
y_pred = clf.predict(X)
print(y_pred)

NameError: name 'clf' is not defined

In [114]:
# Бумагам с низкой ликвидностью даем 3 эшелон
# Добавляем предсказанные значения в DataFrame
df_final_pred = df_final
df_final_pred['Echelon predicted'] = y_pred

# Итерируемся по строкам DataFrame и изменяем значения
for index, row in df_final_pred.iterrows():
    if (row['Liquidity'] == 0):
        df_final_pred.loc[index, 'Echelon predicted'] = 3
        
df_final_pred = df_final_pred.drop_duplicates().reset_index(drop=True)
# Сохраняем DataFrame в Excel
df_final_pred.to_excel('Эшелоны_предикт.xlsx', index=False)

## Можно прочитать готовый датасет

In [27]:
df_final = pd.read_excel('Эшелоны_предикт.xlsx')

In [28]:
df_final

,ISIN,Duration,Amount,Rating,Listing,Liquidity,Currency,Echelon predicted
0,RU000A102CL3,7.146329,1250000000,7,1,0.326191,EUR,1
1,RU000A107D58,3.830634,141112000,5,3,0.465345,USD,2
2,RU000A106V57,1.065374,370000000,5,2,0.331061,AED,2
3,RU000A1080Z9,1.402525,160944000,3,3,0.506891,CNY,3
4,RU000A1082Q4,0.220009,1676729000,7,3,0.571340,USD,1
...,...,...,...,...,...,...,...,...
98,RU000A0JX1S1,0.366681,15000000000,7,3,0.421021,RUB,1
99,RU000A101137,0.300667,25000000000,7,3,0.472665,RUB,1
100,RU000A102DB2,0.339745,20000000000,6,1,0.418488,RUB,1
101,RU000A100VQ6,0.205482,25000000000,7,2,0.510202,RUB,1
